In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from graph_builder import GraphBuilder  # <-- External builder
import pandas as pd
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv,  global_mean_pool, global_add_pool, GraphNorm
from sklearn.model_selection import train_test_split
import ast
from torch_geometric.utils import degree

In [2]:
from GraphBuilder_single_edge import GraphBuilder
from save_model_results import save_model_architecture, append_evaluation_results, evaluate_model

## Model architecture

In [3]:
# Defining a simple GNN model. 
class SimpleGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels,dropout):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = GraphNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = GraphNorm(hidden_channels,hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 2)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch # For multiple graphs in a batch
        x = F.relu(self.norm1(self.conv1(x, edge_index)))
        x = F.relu(self.norm2(self.conv2(x, edge_index)))
        x = global_mean_pool(x, batch)
        return self.lin(x)

In [4]:
# Create the model object.
model = SimpleGNN(in_channels=4, hidden_channels=32,dropout=0.2)

In [5]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=5e-4)

In [6]:
# Generate text file with model architecture
model_results_path = save_model_architecture(model)

## 7-10 loop 

### Data preparation

In [8]:
loop=7

First we read the edges and coefficients of the csv files and save them in lists.

In [9]:
# Create the edge and y lists from the csv files\
edges=[]
y=[]
for i in range(loop,loop+1):
    filename = f'../Graph_Edge_Data/den_graph_data_{loop}.csv'
    df = pd.read_csv(filename)
    edges += df['EDGES'].tolist()
    y += df['COEFFICIENTS'].tolist()
edges = [ast.literal_eval(e) for e in edges]    

In [10]:
# Import the function to add eigenvector features
from torch_geometric.transforms import AddLaplacianEigenvectorPE
eigen_vec= AddLaplacianEigenvectorPE(k=3,attr_name=None)

We need to now translate the edges into dataset forms for training and testing.

In [11]:
# Define the data object through GraphBuilder, then add the eigenvector features
data=[GraphBuilder(solid_edges=x,coeff=y0).build() for x,y0 in zip(edges,y)]
data = [eigen_vec(d) for d in data]

In [12]:
# Split train and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=43)

In [13]:
# Load the data into DataLoader
train_loader = DataLoader(train_data, batch_size=20, shuffle=True)
test_loader = DataLoader(test_data, batch_size=20, shuffle=False)

### Model training

We are interested in graph classification of 0 and 1. We add two graph convolutional layers, making sure that the message passing is extended to two neighbours, and then add graph pooling to average over the whole graph.

In [14]:
def train_model(model, train_loader, test_loader, optimizer, criterion, device, n_epochs=70):
    accuracy_list = []
    loss_list = []
    patience_counter = 0
    patience = 3
    model.to(device)

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            out = model(batch)             # out = model(batch) handles batch.x, batch.edge_index, etc.
            loss = criterion(out, batch.coeff) # Use batch.y (or batch.coeff if that's what your dataset uses)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        loss_list.append(total_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch)
                _, predicted = torch.max(out, 1)
                correct += (predicted == batch.coeff).sum().item()
                total += batch.num_graphs  # graph-level classification

        accuracy = correct / total
        accuracy_list.append(accuracy)

        print(f"Epoch {epoch+1}: Loss={total_loss:.2f}, Accuracy={accuracy:.2f}")

        if epoch>50: 
            if loss_list[epoch-10]-loss_list[epoch] < 0.1:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping") 
                    break   

In [15]:
train_model(model,train_loader,test_loader,optimizer,device='cpu',criterion=criterion)

Epoch 1: Loss=4.93, Accuracy=0.30
Epoch 2: Loss=4.87, Accuracy=0.58
Epoch 3: Loss=4.82, Accuracy=0.70
Epoch 4: Loss=4.77, Accuracy=0.70
Epoch 5: Loss=4.70, Accuracy=0.73
Epoch 6: Loss=4.64, Accuracy=0.70
Epoch 7: Loss=4.59, Accuracy=0.70
Epoch 8: Loss=4.52, Accuracy=0.70
Epoch 9: Loss=4.46, Accuracy=0.70
Epoch 10: Loss=4.39, Accuracy=0.70
Epoch 11: Loss=4.32, Accuracy=0.70
Epoch 12: Loss=4.25, Accuracy=0.70
Epoch 13: Loss=4.30, Accuracy=0.70
Epoch 14: Loss=4.14, Accuracy=0.70
Epoch 15: Loss=4.14, Accuracy=0.70
Epoch 16: Loss=4.15, Accuracy=0.73
Epoch 17: Loss=4.22, Accuracy=0.76
Epoch 18: Loss=4.08, Accuracy=0.73
Epoch 19: Loss=4.14, Accuracy=0.73
Epoch 20: Loss=4.13, Accuracy=0.73
Epoch 21: Loss=4.23, Accuracy=0.73
Epoch 22: Loss=4.08, Accuracy=0.73
Epoch 23: Loss=4.15, Accuracy=0.73
Epoch 24: Loss=4.26, Accuracy=0.73
Epoch 25: Loss=4.15, Accuracy=0.64
Epoch 26: Loss=4.14, Accuracy=0.67
Epoch 27: Loss=4.08, Accuracy=0.67
Epoch 28: Loss=4.12, Accuracy=0.67
Epoch 29: Loss=4.28, Accuracy

### Model evaluation

We evaluate the model on test set in terms of Accuracy, Precision, Recall and F1 score. Then we save the results in a txt file. Then we also print the same metrics for the training set.

In [16]:
evaluation =evaluate_model(model, test_loader, device='cpu')

AttributeError: 'SimpleGNN' object has no attribute 'epoch'

In [17]:
append_evaluation_results(model_results_path, evaluation, loop = loop )

NameError: name 'evaluation' is not defined

In [48]:
evaluation_train =evaluate_model(model, train_loader, device='cpu')
print(evaluation_train)

{'Accuracy': 0.8059314361220544, 'Precision': array([0.85554778, 0.70786599]), 'Recall': array([0.85268841, 0.7125889 ]), 'F1 Score': array([0.8541157 , 0.71021959]), 'Confusion Matrix': array([[69651, 12033],
       [11760, 29157]])}
